In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [4]:
class DatabaseHandler:
    @staticmethod
    def create_database():
        conn = sqlite3.connect('country_data.db')
        conn.close()
    

    @staticmethod
    def create_table():
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()
        #create a table with the country, artist, their song, and their rank in that country
        c.execute('''CREATE TABLE IF NOT EXISTS songs (
                    country TEXT,
                    artist_name TEXT,
                    song_name TEXT,
                    rank INTEGER)''')
        conn.commit()
        conn.close()

    @staticmethod
    def insert_data(country, artist_name, song_name, rank):
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()
        #insert country and artist info into the database
        c.execute("INSERT INTO songs (country, rank, song_name, artist_name) VALUES (?, ?, ?, ?)", (country, artist_name, song_name, rank))
        conn.commit()
        conn.close()

    def populate_db(df = pd.read_csv('country_charts.csv')):
        DatabaseHandler.create_database()
        DatabaseHandler.create_table()

        for index, row in df.iterrows():
            DatabaseHandler.insert_data(row['country'], row['artist_name'], row['song_title'], row['Pos'])

    #function to find songs that are unique to that country
    def find_unique_songs(country):
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()

        c.execute("SELECT DISTINCT song_name FROM songs WHERE country=?", (country,))
        unique_songs = c.fetchall()
        conn.close()
        return unique_songs

DatabaseHandler.populate_db()
